In [ ]:
!pip install gymnasium
!pip install d3rlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.3/166.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 30.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827619 sha256=e7f47a2f501d0902555b621edecdf9850993b21c9d495ab4dd362cb862ca504f
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does n

In [ ]:
import json
import dataclasses
import gymnasium as gym
import numpy as np
import d3rlpy
import pandas as pd
import torch
import torch.nn as nn

In [ ]:
# utilities
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

# visualization
#import matplotlib.pyplot as plt
#%matplotlib inline

#plt.figure(figsize=(12,8))
#xspace = np.linspace(0, 10, 100)
#yspace = np.linspace(0, 10, 100)
#plt.xlabel('xxx')
#plt.ylabel('yyy')
#plt.plot(xspace, yspace)

Visualize it.

In [ ]:
preamble = 'drive/MyDrive/data'
datafile = 'sample.json'

print(datafile)

#df1 = pd.read_json(preamble + '/' + datafile)

#print(df1.index)
#print(df1.columns)
#print(df1.info)

#ob_data = df1['observations']

#print(ob_data.info)

#ob_array = ob_data.to_numpy()
#print(ob_array[1])

f = open(preamble + '/' + datafile)

d = json.load(f)

f.close()

print(d.keys())

#for i in d['bandwidth_predictions']:
#    print(i)

bwe_data = np.array(d['bandwidth_predictions'])

print(bwe_data)
print(bwe_data.shape)

#for j in d['observations']:
#  print(j)

ob_data = np.array(d['observations'])
print(ob_data)

print(ob_data[:,2])

#kk = np.array([1,2,3,4,5,6])

#ss = kk.reshape((2,3))

#print(ss)

#mm = ss.reshape((6,))
#print(mm)

sample.json
dict_keys(['policy_id', 'observations', 'bandwidth_predictions', 'video_quality', 'audio_quality'])
[  20000.   20000.   20000. ... 7876328. 7873648. 7856528.]
(3642,)
[[  73600.               0.               0.         ...       0.
        0.               0.        ]
 [  79866.66666667   58533.33333333   15066.66666667 ...       0.
        0.               0.        ]
 [  79866.66666667   59600.           34000.         ...       0.
        0.               0.        ]
 ...
 [1219200.         2152000.         1771733.33333333 ...       0.
        0.               0.        ]
 [  82400.         1219600.         2130000.         ...       0.
        0.               0.        ]
 [  62666.66666667   60800.         1219200.         ...       0.
        0.               0.        ]]
[      0.           15066.66666667   34000.         ... 1771733.33333333
 2130000.         1219200.        ]


The observation vector at a time step n encapsulates observed network statistics that characterize the state of the bottleneck link between the sender and receiver over the 5 most recent short term monitor intervals (MI) of 60ms and the 5 most recent long-term MIs of 600ms. Specifically, the observation vector tracks 15 different network features over 5 short and 5 long term MIs (15 features x (5 short term MIs + 5 long term MIs) = 150). The 15 features and their description are as follows. Features are based on packets received during the short and long term monitor intervals.

    1. Receiving rate: rate at which the client receives data from the sender during a MI, unit: bps.
    2. Number of received packets: total number of packets received in a MI, unit: packet.
    3. Received bytes: total number of bytes received in a MI, unit: Bytes.
    4. Queuing delay: average delay of packets received in a MI minus the minimum packet delay observed so far, unit: ms.
    5. Delay: average delay of packets received in a MI minus a fixed base delay of 200ms, unit: ms.
    6. Minimum seen delay: minimum packet delay observed so far, unit: ms.
    7. Delay ratio: average delay of packets received in a MI divided by the minimum delay of packets received in the same MI, unit: ms/ms.
    8. Delay average minimum difference: average delay of packets received in a MI minus the minimum delay of packets received in the same MI, unit: ms.
    9. Packet interarrival time: mean interarrival time of packets received in a MI, unit: ms.
    10. Packet jitter: standard deviation of interarrival time of packets received in a MI, unit: ms.
    11. Packet loss ratio: probability of packet loss in a MI, unit: packet/packet.
    12. Average number of lost packets: average number of lost packets given a loss occurs, unit: packet.
    13. Video packets probability: proportion of video packets in the packets received in a MI, unit: packet/packet.
    14. Audio packets probability: proportion of audio packets in the packets received in a MI, unit: packet/packet.
    15. Probing packets probability: proportion of probing packets in the packets received in a MI, unit: packet/packet.

 The indices (zero-indexed) of features over the 5 short term MIs are {(feature number - 1) * 10, ., ., ., (feature number - 1) * 10 + 4}.

The indices (zero-indexed) of features over the 5 long term MIs are {(feature number - 1) * 10 + 5, ., ., ., (feature number - 1) * 10 + 9}.

In [ ]:
# enumeration for features numbering
from enum import IntEnum

class Feature(IntEnum):
    RECV_RATE           = 1
    RECV_PKT_AMOUNT     = 2
    RECV_BYTEs          = 3
    QUEUING_DELAY       = 4
    DELAY               = 5
    MIN_SEEN_DELAY      = 6
    DELAY_RATIO         = 7
    DELAY_MIN_DIFF      = 8
    PKT_INTERARRIVAL    = 9
    PKT_JITTER          = 10
    PKT_LOSS_RATIO      = 11
    PKT_AVE_LOSS        = 12
    VIDEO_PKT_PROB      = 13
    AUDIO_PKT_PROB      = 14
    PROB_PKT_PROB       = 15

In [ ]:
# reward function {0.6 * ln(4R + 1) -D - 10L} (inspired from R3Net)
# R - receive rate at this time step, mb/s
# D - queuing delay, ms (R3Net uses average Round Trip Time)
# L - packet loss rate
# receiving more packets is rewarded, delay and packet loss is penalized.

def reward_r3net (feature_vec):
  # use the 5 recent short MIs. (TODO: refinement)
  receive_rate = np.sum(feature_vec[(Feature.RECV_RATE-1)*10:(Feature.RECV_RATE-1)*10+4])/5
  queuing_delay = np.sum(feature_vec[(Feature.QUEUING_DELAY-1)*10:(Feature.QUEUING_DELAY-1)*10+4])/5
  pkt_loss_rate = np.sum(feature_vec[(Feature.PKT_LOSS_RATIO-1)*10:(Feature.PKT_LOSS_RATIO-1)*10+4])/5

  return (0.6 * np.log(4*receive_rate + 1) - queuing_delay/1000 - 10*pkt_loss_rate)


In [ ]:
# reward function (inspired from OnRL)
# definition from OnRL follows
# r_t = alpha * sum_{n=1}^{N} q_n
#       - beta * sum_{n=1}^{N} l_n
#       - eta * sum_{n=1}^{N} d_n
#       - phi * sum_{n=1}^{N-1} |q_n - q_{n-1}|
# N - number of RTP packets in a state
# q_n - receive rate at this time step, mb/s
# l_n - packet loss rate
# d_n - delay, ms
# |q_n - q_{n-1}| - receive rates difference between two steps. ()
# alpha, beta, eta and phi are hyperparameters impacting training effectiveness.
#
# Here we uses the statistic of recent measure intervals (as the feature vectors)
#

def reward_onrl (feature_vec):
  alpha = 50
  beta = 50
  eta = 10
  phi = 20

  receive_rate = np.sum(feature_vec[(Feature.RECV_RATE-1)*10:(Feature.RECV_RATE-1)*10+4])
  queuing_delay = np.sum(feature_vec[(Feature.QUEUING_DELAY-1)*10:(Feature.QUEUING_DELAY-1)*10+4])
  pkt_loss_rate = np.sum(feature_vec[(Feature.PKT_LOSS_RATIO-1)*10:(Feature.PKT_LOSS_RATIO-1)*10+4])

  # incomplete - the last term is not in place. (TODO)
  return (alpha * receive_rate - beta * pkt_loss_rate - eta * queuing_delay - 0)

In [ ]:
# create a dataset

# observations (same as the features vector)
observations = ob_data

# actions: bandwidth predictions in the dataset
actions = bwe_data

# rewards
rewards = np.zeros(len(ob_data))
for i, o in enumerate(list(ob_data)):
  rewards[i] = reward_r3net(o)

print(rewards.shape)
print(rewards[0:5])

# terminal flags
terminals = np.random.randint(2, size=len(actions))

# offline learning dataset
dataset = d3rlpy.dataset.MDPDataset(
    observations=observations,
    actions=actions,
    rewards=rewards,
    terminals=terminals,
)

(3642,)
[  0.         -22.58180982 -22.57097435 -37.22612335 -37.22612335]
2023-11-19 20:46:34 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(150,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2023-11-19 20:46:34 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2023-11-19 20:46:34 [info     ] Action size has been automatically determined. action_size=7887689


Create a discrete action space

In [ ]:
def make_bw_act_space(actions):
  low = 0
  #high = max(actions)
  high = np.inf

  shape = (len(actions),)
  #print(f"action shape {shape} \n")
  bw_action_space = gym.spaces.Box(low, high, shape, dtype=np.float32)
  return bw_action_space

Create a continuous state space (a state is an observation feature)

In [ ]:
def make_bw_obs_space(observations):
    low = 0.0
    high = np.inf
    shape = (len(observations),len(observations[0]))
    #print(f"state space shape {shape} \n")
    return gym.spaces.Box(low, high, shape, dtype=np.float32)

Create the environment for bandwidth estimation.

In [ ]:
class BandwidthEstimation(gym.Env):
    def __init__(self, max_steps=1000):
        self.actions = actions
        self.observations = observations
        self.action_space = make_bw_act_space(actions)
        self.observation_space = make_bw_obs_space(observations)
        self.seed=0
        self.log = ''
        self.max_steps = len(actions)
        self.cur_step=0

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.steps_left = self.max_steps
        self.cur_step = 0

        return self.observations[self.cur_step]

    def step(self, action):
        # Do selected action (TODO)
        cur_estimate = self.actions[self.cur_step]
        cur_observation = self.observations[self.cur_step]
        self.log += f'bwe: {cur_estimate}\n'

        # Calculate the reward (TODO)
        reward = reward_r3net(cur_observation);

        self.log += str(self.state) + '\n'

        self.steps_left -= 1
        done = (self.steps_left <= 0)

        # step count increases
        self.cur_step += 1

        return cur_observation, reward, done, {}

    def close(self):
        pass

    def render(self, mode=None):
        print(self.log)
        self.log = ''

In [ ]:
# to be used for online learning (or evaluation)
bwe_env = BandwidthEstimation()
bwe_env.reset()

action shape (3642,) 

state space shape (3642, 150) 



array([ 7.36000000e+04,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  7.36000000e+03,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  4.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  4.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        5.52000000e+02,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  5.52000000e+02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  1.50000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.50000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -2.50000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -2.50000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  1.96000000e+02,  0.00000000e+00,
        0.00000000e+00,  

We create a customized deep network for DRL.

In [ ]:
## incomplete.  TODO
#

class LTSMEncoder(nn.Module):
    def __init__(self, observation_shape, feature_size):
        super().__init__()
        self.feature_size = feature_size
        self.fc1 = nn.Linear(observation_shape[0], feature_size)
        self.fc2 = nn.Linear(feature_size, feature_size)

    def forward(self, x):
        h = torch.relu(self.fc1(x))
        h = torch.relu(self.fc2(h))
        return h

@dataclasses.dataclass()
class LTSMEncoderFactory(d3rlpy.models.EncoderFactory):
    feature_size: int

    def create(self, observation_shape):
        return LTSMEncoder(observation_shape, self.feature_size)

    @staticmethod
    def get_type() -> str:
        return "ltsm"

To test this environment, we use d3rlpy library to build a DQN agent that uses a Q-network.

Option 1: Create a DQN model.

In [ ]:
# encoder factory
encoder = d3rlpy.models.VectorEncoderFactory(
    hidden_units=[64, 32],
    activation='tanh',
)


# setup algorithm
dqn = d3rlpy.algos.DQNConfig(
    batch_size=32,
    learning_rate=2.5e-4,
    target_update_interval=100,
    encoder_factory = encoder).create(device='cpu:1')

# start offline training
dqn.fit(
   dataset,
   n_steps=1000,
   n_steps_per_epoch=10000,
   evaluators={
       "environment": d3rlpy.metrics.EnvironmentEvaluator(bwe_env),
   },
)

2023-11-19 20:16:28 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(150,)]), action_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=7887689)
2023-11-19 20:16:28 [info     ] Directory is created at d3rlpy_logs/DQN_20231119201628
2023-11-19 20:16:28 [debug    ] Building models...
2023-11-19 20:16:41 [debug    ] Models have been built.
2023-11-19 20:16:41 [info     ] Parameters                     params={'observation_shape': [150], 'action_size': 7887689, 'config': {'type': 'dqn', 'params': {'batch_size': 32, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'learning_rate': 0.00025, 'optim_factory': {'type': 'adam', 'params': {'betas': [0.9, 0.999], 'eps': 

[]

Option 2: Create a Conservative Q-learning model.

In [ ]:
# encoder factory
actor_encoder = d3rlpy.models.VectorEncoderFactory(
    hidden_units=[64, 32],
    activation='tanh',
)

critic_encoder = d3rlpy.models.VectorEncoderFactory(
    hidden_units=[64, 32],
    activation='tanh',
)

## configuration for a Conservative Q-lerning model.
cql_conf = d3rlpy.algos.DiscreteCQLConfig(
    batch_size=32,
    gamma=0.99,
    observation_scaler=None,
    action_scaler=None,
    reward_scaler=None,
    learning_rate=6.25e-05,
    encoder_factory=actor_encoder,
    n_critics=1,
    target_update_interval=8000,
    alpha=1.0)

cql = d3rlpy.algos.DiscreteCQL(cql_conf, device='cpu:1')

##
# offline training
cql.fit(
    dataset,
    n_steps=10000,
    n_steps_per_epoch=1000,

    evaluators={
        'environment': d3rlpy.metrics.EnvironmentEvaluator(bwe_env),
    },
)

2023-11-19 14:34:33 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(150,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=7887689)
2023-11-19 14:34:33 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20231119143433
2023-11-19 14:34:33 [debug    ] Building models...
2023-11-19 14:34:43 [debug    ] Models have been built.
2023-11-19 14:34:43 [info     ] Parameters                     params={'observation_shape': [150], 'action_size': 7887689, 'config': {'type': 'discrete_cql', 'params': {'batch_size': 32, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'learning_rate': 6.25e-05, 'optim_factory': {'type': 'adam', 'params': {'betas': [0.

Epoch 1/10:   0%|          | 0/1000 [00:00<?, ?it/s]

Transfer to DQN for online learning

In [ ]:
# Incomplete. TODO
#
#

## transfer to online learning using DQN

# setup experience replay buffer
buffer = d3rlpy.dataset.create_fifo_replay_buffer(limit=100000, env=bwe_env)

# setup exploration strategy if necessary
explorer = d3rlpy.algos.ConstantEpsilonGreedy(0.1)

# transfer to a new DQN
new_dqn = d3rlpy.algos.DQNConfig().create()
new_dqn.build_with_env(bwe_env)
new_dqn.copy_q_function_from(cql)

# start finetuning
new_dqn.fit_online(bwe_env, buffer, explorer, n_steps=100000)